<h1><center>IE 306 - Project 1<br>Simulation with SimPy<br>

# Group 1

1. Muhammed Mustafa Alparslan - 2009400189
2. Baran Deniz Korkmaz - 2015400183
3. Doğukan Kalkan - 2015400132

In [1]:
import simpy
import random
import numpy as np
import math

# Distribution Related Parameters
All the variables required for defining probability distributions for events initialized at the start. 


In [2]:
INTERARRIVAL_MEAN = 6.0
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

AAM_SERVICE_TIME_MEAN = 5.0
AAM_SERVICE_TIME_RATE = 1.0 / AAM_SERVICE_TIME_MEAN

OPERATOR_1_SERVICE_TIME_MEAN = 12.0
OPERATOR_1_SERVICE_TIME_STDEV = 6.0

QUEUE_WAITING_TIME_LIMIT = 10.0

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60.0

CUSTOMERS_SERVICED = 0


# System Variables
Service time arrays are defined for both the caller and operators separately.
Operators have different waiting queues and unsatisfied customers from callers and operators are stored separately.

In [3]:
service_times_aam = []
service_times_operator_1 = []
service_times_operator_2 = []
queue_waiting_times_operator_1 = []
queue_waiting_times_operator_2 = []
unsatisfied_customers_incorrect_routed = []
unsatisfied_customers_overwaiting = []

# Customer
The main entity in the system is the customer. Every customer is defined with a name, arrival time, initial caller action, operator to be forwarded, and stopping event. Actions follow the flow logic described above. We will chain the actions as caller, forwarding, operator queue, and serving. 

After a customer is created it will run the `call` function to process the caller action. If all callers are busy then the call will be dropped immediately. Otherwise a random service time will be assigned and added to the service array. 

Next step when the caller identifies the customer and its operator we create a uniform random variable for routing. Having the random lower than 0.1 means there is an error and the customer is routed to the wrong operator. In this case this customer is added to the unsatisfied customer list from routing.

If the routing was successful the customer will be redirected to the related operator by calling respective functions.

Even though operator functions are quite similar we kept them separate to make it easier to read. Operator drops the customers which are waiting for too long from the queue before serving a new one. Queue waiting is defined as a timeout that triggers an event ( as not triggering serving) after ten minutes. Any customer who reaches the timeout will be dropped and added to unsatisfied customer list.

When a customer comes to the operator from the queue, a uniformly distributed random variable is created for service time. It is also added to the service time list of the operator. After the serving time  is completed we increase the number of customers served and check if the total number reached the sample size. In that case the simulation is completed and the stopping event will be triggered.


In [4]:
#Arriving customers are initialized.
class Customer(object):
    def __init__(self, name, env, opr_num, sample_size, stopping_event):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.opr_num = opr_num
        self.sample_size = sample_size
        self.stopping_event = stopping_event
    
    #Customers call automated answering mechanism.
    def call(self):
        global CUSTOMERS_SERVICED
        with automated_answer_mech.request() as req:
            # If the automated answer mechanism is full, the customer is dropped from the system.
            if automated_answer_mech.count == automated_answer_mech.capacity:
                return
            aam_service_duration = random.expovariate(AAM_SERVICE_TIME_RATE)
            yield self.env.timeout(aam_service_duration)
            automated_answer_mech.release(req)
            service_times_aam.append(aam_service_duration)
            
            # The misrouted customers with a probability of 0.1 are dropped.
            incorrect_routing_probability = random.uniform(0, 1)
            if incorrect_routing_probability < 0.1:
                unsatisfied_customers_incorrect_routed.append(self.name)
                return
            
            # Correctly routed customers are redirected into the operators.
            if self.opr_num == 1:
                yield self.env.process(self.call_operator_1())
            else:
                yield self.env.process(self.call_operator_2())
    
    # The customers that want to use operator 1 are connected.
    def call_operator_1(self):
        global CUSTOMERS_SERVICED
        queue_start_time = self.env.now
        with operator_1.request(priority = 0) as req:
            yield req | self.env.timeout(QUEUE_WAITING_TIME_LIMIT)
            # The customers that wait more than 10 minutes are dropped.
            if not req.triggered:
                queue_waiting_times_operator_1.append(QUEUE_WAITING_TIME_LIMIT)
                unsatisfied_customers_overwaiting.append(self.name)
                return

            queue_waiting_time = self.env.now - queue_start_time
            if queue_waiting_time != 0:
                queue_waiting_times_operator_1.append(queue_waiting_time)

            operator_1_service_duration = math.log(np.random.lognormal(OPERATOR_1_SERVICE_TIME_MEAN, OPERATOR_1_SERVICE_TIME_STDEV), math.e)
            while operator_1_service_duration < 0:
                operator_1_service_duration = math.log(np.random.lognormal(OPERATOR_1_SERVICE_TIME_MEAN, OPERATOR_1_SERVICE_TIME_STDEV), math.e)

            yield self.env.timeout(operator_1_service_duration)
            service_times_operator_1.append(operator_1_service_duration)
            CUSTOMERS_SERVICED += 1
            # If the customer serviced is the last customer, then the stopping event that terminates the simulation is triggered.
            if CUSTOMERS_SERVICED == self.sample_size:
                self.stopping_event.succeed()

    # The customers that want to use operator 2 are connected.
    def call_operator_2(self):
        global CUSTOMERS_SERVICED
        queue_start_time = self.env.now
        with operator_2.request(priority = 0) as req:
            yield req | self.env.timeout(QUEUE_WAITING_TIME_LIMIT)
            # The customers that wait more than 10 minutes are dropped.
            if not req.triggered:
                queue_waiting_times_operator_2.append(QUEUE_WAITING_TIME_LIMIT)
                unsatisfied_customers_overwaiting.append(self.name)
                return

            queue_waiting_time = self.env.now - queue_start_time
            if queue_waiting_time != 0:
                queue_waiting_times_operator_2.append(queue_waiting_time)

            operator_2_service_duration = random.uniform(1,7)
            yield self.env.timeout(operator_2_service_duration)
            service_times_operator_2.append(operator_2_service_duration)
            CUSTOMERS_SERVICED += 1
            # If the customer serviced is the last customer, then the 
            # stopping event that terminates the simulation is triggered.
            if CUSTOMERS_SERVICED == self.sample_size:
                self.stopping_event.succeed()

## Creating customers and breaks
Customers are generated with interarrival times for the caller service. After creating a caller serving time we assign the operator depending on the uniformly distributed random variable. 

For the operators break times generated by a poisson distribution. Priority is given as one for breaks and zero for customer serving events. This way we guarantee that all the customers will be served before operators taking a break since they have a higher priority.


In [5]:
def customer_generator(env, sample_size, stopping_event):
    while True:
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        probability = random.uniform(0,1)
        opr_num = 1 if probability <= 0.3 else 2
        customer = Customer('Cust %s' %(i+1), env, opr_num, sample_size, stopping_event)


def break_call(env, operator):
    while True:
        yield env.timeout(np.random.poisson(BREAK_TIME_RATE))
        with operator.request(priority = 1) as req:
            yield req
            yield env.timeout(BREAK_TIME)

## Seeds and Running Env
We used ten different seeds and two different customer numbers as 1000 and 5000. For each seed value a new random is created to be used in the simulation.

Using the simpy environment we defined the stopping event, resources for callers and operators. Default provided capacity parameters are used as  100 for callers and 1 for operators. At last we define the processes defined above as parameters to the environment for actions and run it until the stopping event.


In [6]:
SEED_LIST = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CUSTOMER_SIZE_LIST = [1000, 5000]

UTILIZATION_AAM_FINAL = [[], []]
UTILIZATION_OPERATOR_1_FINAL = [[], []]
UTILIZATION_OPERATOR_2_FINAL = [[], []]
AVERAGE_TOTAL_WAITING_TIME_FINAL = [[], []]
TOTAL_WAITING_TIME_TO_TOTAL_SYSTEM_TIME_FINAL = [[], []]
NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_1_FINAL = [[], []]
NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_2_FINAL = [[], []]
NUMBER_OF_UNSATISFIED_CUSTOMERS_FINAL = [[], []]


for seed in SEED_LIST:
    random.seed(seed)
    np.random.seed(seed)
    for (i, current_size) in enumerate(CUSTOMER_SIZE_LIST):
        CUSTOMERS_SERVICED = 0
        service_times_aam.clear()
        service_times_operator_1.clear()
        service_times_operator_2.clear()
        queue_waiting_times_operator_1.clear()
        queue_waiting_times_operator_2.clear()
        unsatisfied_customers_incorrect_routed.clear()
        unsatisfied_customers_overwaiting.clear()

        env = simpy.Environment()
        stopping_event = env.event()
        automated_answer_mech = simpy.Resource(env, capacity=100)
        operator_1 = simpy.PriorityResource(env, capacity=1)
        operator_2 = simpy.PriorityResource(env, capacity=1)
        env.process(customer_generator(env, current_size, stopping_event))
        env.process(break_call(env, operator_1))
        env.process(break_call(env, operator_2))
        env.run(until=stopping_event)
        system_time = env.now

        print('System Time for System Size %d: %g' %(current_size,system_time))

        ###COLLECTING STATISTICS
        # UTILIZATION OF THE AAM
        utilization_aam = (sum(service_times_aam) / (100 * system_time))
        # UTILIZATION OF THE OPERATORS
        utilization_operator_1 = (sum(service_times_operator_1) / system_time)
        utilization_operator_2 = (sum(service_times_operator_2) / system_time)
        # AVERAGE TOTAL WAITING TIME
        total_waiting_time = sum(queue_waiting_times_operator_1) + sum(queue_waiting_times_operator_2)
        average_total_waiting_time = total_waiting_time / current_size
        # Maximum Total Waiting Time to Total System Time Ratio
        total_waiting_time_to_total_system_time = total_waiting_time / system_time
        #Number of waiting customers
        number_of_waiting_customers_operator_1 = len(queue_waiting_times_operator_1)
        number_of_waiting_customers_operator_2 = len(queue_waiting_times_operator_2)
        # Unsatisfied Customers
        number_of_unsatisfied_customers = len(unsatisfied_customers_overwaiting) + len(unsatisfied_customers_incorrect_routed)

        UTILIZATION_AAM_FINAL[i].append(utilization_aam)
        UTILIZATION_OPERATOR_1_FINAL[i].append(utilization_operator_1)
        UTILIZATION_OPERATOR_2_FINAL[i].append(utilization_operator_2)
        AVERAGE_TOTAL_WAITING_TIME_FINAL[i].append(average_total_waiting_time)
        TOTAL_WAITING_TIME_TO_TOTAL_SYSTEM_TIME_FINAL[i].append(total_waiting_time_to_total_system_time)
        NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_1_FINAL[i].append(number_of_waiting_customers_operator_1)
        NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_2_FINAL[i].append(number_of_waiting_customers_operator_2)
        NUMBER_OF_UNSATISFIED_CUSTOMERS_FINAL[i].append(number_of_unsatisfied_customers)

System Time for System Size 1000: 7447.56
System Time for System Size 5000: 36311.9
System Time for System Size 1000: 7117.02
System Time for System Size 5000: 36728.8
System Time for System Size 1000: 7161.45
System Time for System Size 5000: 36751.5
System Time for System Size 1000: 7130.46
System Time for System Size 5000: 35424.6
System Time for System Size 1000: 7521.97
System Time for System Size 5000: 35817.1
System Time for System Size 1000: 7640.47
System Time for System Size 5000: 36064.1
System Time for System Size 1000: 7384.18
System Time for System Size 5000: 36083
System Time for System Size 1000: 7335.88
System Time for System Size 5000: 36787.1
System Time for System Size 1000: 6831.8
System Time for System Size 5000: 36093.3
System Time for System Size 1000: 7373.41
System Time for System Size 5000: 35300.8


## Statistics

The final part is collecting statistics from the data we collected in the simulation. Every variable is appended to the corresponding lists and we will take the average values from these lists while calculating the outputs. 


In [7]:
#FINAL STATISTICS:

#1.SAMPLE_SIZE = 1000
utilization_aam_1000 = np.average(UTILIZATION_AAM_FINAL[0])
utilization_operator_1_1000 = np.average(UTILIZATION_OPERATOR_1_FINAL[0])
utilization_operator_2_1000 = np.average(UTILIZATION_OPERATOR_2_FINAL[0])
average_total_waiting_time_1000 = np.average(AVERAGE_TOTAL_WAITING_TIME_FINAL[0])
total_waiting_time_to_total_system_time_1000 = np.max(TOTAL_WAITING_TIME_TO_TOTAL_SYSTEM_TIME_FINAL[0])
average_number_of_waiting_customers_operator_1_1000 = np.average(NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_1_FINAL[0])
average_number_of_waiting_customers_operator_2_1000 = np.average(NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_2_FINAL[0])
average_number_of_unsatisfied_customers_1000 = np.average(NUMBER_OF_UNSATISFIED_CUSTOMERS_FINAL[0])

#2.SAMPLE_SIZE = 5000
utilization_aam_5000 = np.average(UTILIZATION_AAM_FINAL[1])
utilization_operator_1_5000 = np.average(UTILIZATION_OPERATOR_1_FINAL[1])
utilization_operator_2_5000 = np.average(UTILIZATION_OPERATOR_2_FINAL[1])
average_total_waiting_time_5000 = np.average(AVERAGE_TOTAL_WAITING_TIME_FINAL[1])
total_waiting_time_to_total_system_time_5000 = np.average(TOTAL_WAITING_TIME_TO_TOTAL_SYSTEM_TIME_FINAL[1])
average_number_of_waiting_customers_operator_1_5000 = np.average(NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_1_FINAL[1])
average_number_of_waiting_customers_operator_2_5000 = np.average(NUMBER_OF_WAITING_CUSTOMERS_OPERATOR_2_FINAL[1])
average_number_of_unsatisfied_customers_5000 = np.average(NUMBER_OF_UNSATISFIED_CUSTOMERS_FINAL[1])

print('Simulation of the System for 1000 Customers:')
print("Utillization of Answering System: ",utilization_aam_1000)
print("Utillization of Operator 1: ",utilization_operator_1_1000)
print("Utillization of Operator 2: ",utilization_operator_2_1000)
print("Average Total Waiting Time: ",average_total_waiting_time_1000)
print("Maximum Total Waiting Time to Total System Time Ratio: ",total_waiting_time_to_total_system_time_1000)
print("Average Number of People Waiting to be Served by Operator 1: ",average_number_of_waiting_customers_operator_1_1000)
print("Average Number of People Waiting to be Served by Operator 2: ",average_number_of_waiting_customers_operator_2_1000)
print("Average Number of Unsatistied Customers: ",average_number_of_unsatisfied_customers_1000,"\n")

print('Simulation of the System for 5000 Customers:')
print("Utillization of Answering System:",utilization_aam_5000)
print("Utillization of Operator 1: ",utilization_operator_1_5000)
print("Utillization of Operator 2: ",utilization_operator_2_5000)
print("Average Total Waiting Time: ",average_total_waiting_time_5000)
print("Maximum Total Waiting Time to Total System Time Ratio: ",total_waiting_time_to_total_system_time_5000)
print("Average Number of People Waiting to be Served by Operator 1: ",average_number_of_waiting_customers_operator_1_5000)
print("Average Number of People Waiting to be Served by Operator 2: ",average_number_of_waiting_customers_operator_2_5000)
print("Average Number of Unsatistied Customers: ",average_number_of_unsatisfied_customers_5000)

Simulation of the System for 1000 Customers:
Utillization of Answering System:  0.008168507423859937
Utillization of Operator 1:  0.44230979895347133
Utillization of Operator 2:  0.40536237756685145
Average Total Waiting Time:  3.5782916099952558
Maximum Total Waiting Time to Total System Time Ratio:  0.5498132061287802
Average Number of People Waiting to be Served by Operator 1:  328.3
Average Number of People Waiting to be Served by Operator 2:  753.5
Average Number of Unsatistied Customers:  209.4 

Simulation of the System for 5000 Customers:
Utillization of Answering System: 0.008356072314241769
Utillization of Operator 1:  0.44620337247506814
Utillization of Operator 2:  0.40829447705081223
Average Total Waiting Time:  3.6350333551253002
Maximum Total Waiting Time to Total System Time Ratio:  0.5031171573551524
Average Number of People Waiting to be Served by Operator 1:  1647.2
Average Number of People Waiting to be Served by Operator 2:  3777.9
Average Number of Unsatistied Cus

## Results
The collection of statistics leads us into the derivation of results by observing the data. For both sizes, we observe that the utilization of system servers are nearly the same. The average waiting times also remain unchanged, except a minor decrease in maximum total waiting time to total system time ratio. Plus, we see that there exists a linear relationship by the factor of 5 between the system size and the average number of people waiting to be served and unsatisfied customers. We can conclude that as the system size grows, the long-run measures related to taking the average show a linear growth as well, while utilization measures remain unchanged. This implies that the system size has nearly no effect in system performance.
